In [ ]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [ ]:
Train_dir = 'train'
Test_dir = 'test'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in sorted(os.listdir(dir)):
        label_path = os.path.join(dir, label)
        if not os.path.isdir(label_path):
            continue  # skip files like .DS_Store
        for imagename in os.listdir(label_path):
            image_paths.append(os.path.join(label_path, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.zip to train.zip


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.zip to test.zip


In [ ]:
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/angry/15612.jpg   
  inflating: __MACOSX/train/angry/._15612.jpg  
  inflating: train/angry/29855.jpg   
  inflating: __MACOSX/train/angry/._29855.jpg  
  inflating: train/angry/20050.jpg   
  inflating: __MACOSX/train/angry/._20050.jpg  
  inflating: train/angry/22647.jpg   
  inflating: __MACOSX/train/angry/._22647.jpg  
  inflating: train/angry/1863.jpg    
  inflating: __MACOSX/train/angry/._1863.jpg  
  inflating: train/angry/22121.jpg   
  inflating: __MACOSX/train/angry/._22121.jpg  
  inflating: train/angry/27059.jpg   
  inflating: __MACOSX/train/angry/._27059.jpg  
  inflating: train/angry/16290.jpg   
  inflating: __MACOSX/train/angry/._16290.jpg  
  inflating: train/angry/29060.jpg   
  inflating: __MACOSX/train/angry/._29060.jpg  
  inflating: train/angry/33986.jpg   
  inflating: __MACOSX/train/angry/._33986.jpg  
  inflating: train/angry/22653.jpg   
  inflating: __MACOSX/train/angry/._22653.jpg  
  in

In [ ]:
!unzip test.zip

Streaming output truncated to the last 5000 lines.
  inflating: test/surprise/31961.jpg  
  inflating: __MACOSX/test/surprise/._31961.jpg  
  inflating: test/surprise/29690.jpg  
  inflating: __MACOSX/test/surprise/._29690.jpg  
  inflating: test/surprise/22921.jpg  
  inflating: __MACOSX/test/surprise/._22921.jpg  
  inflating: test/surprise/22666.jpg  
  inflating: __MACOSX/test/surprise/._22666.jpg  
  inflating: test/surprise/34004.jpg  
  inflating: __MACOSX/test/surprise/._34004.jpg  
  inflating: test/surprise/6877.jpg  
  inflating: __MACOSX/test/surprise/._6877.jpg  
  inflating: test/surprise/34986.jpg  
  inflating: __MACOSX/test/surprise/._34986.jpg  
  inflating: test/surprise/4912.jpg  
  inflating: __MACOSX/test/surprise/._4912.jpg  
  inflating: test/surprise/22128.jpg  
  inflating: __MACOSX/test/surprise/._22128.jpg  
  inflating: test/surprise/7957.jpg  
  inflating: __MACOSX/test/surprise/._7957.jpg  
  inflating: test/surprise/35118.jpg  
  inflating: __MACOSX/test

In [ ]:
Train_dir = 'train'
Test_dir = 'test'

In [ ]:
train = pd.DataFrame()
train["image"], train["label"] = createdataframe(Train_dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [ ]:
print(train)

                          image     label
0         train/angry/34627.jpg     angry
1         train/angry/33433.jpg     angry
2         train/angry/27863.jpg     angry
3         train/angry/14142.jpg     angry
4         train/angry/18880.jpg     angry
...                         ...       ...
28816  train/surprise/15747.jpg  surprise
28817  train/surprise/10213.jpg  surprise
28818  train/surprise/28829.jpg  surprise
28819  train/surprise/30223.jpg  surprise
28820  train/surprise/16074.jpg  surprise

[28821 rows x 2 columns]


In [ ]:
test = pd.DataFrame()
test["image"], test["label"] = createdataframe(Test_dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [ ]:
print(test)

                        image     label
0        test/angry/17452.jpg     angry
1        test/angry/29730.jpg     angry
2        test/angry/26572.jpg     angry
3        test/angry/21131.jpg     angry
4         test/angry/2484.jpg     angry
...                       ...       ...
7061   test/surprise/4243.jpg  surprise
7062  test/surprise/26910.jpg  surprise
7063  test/surprise/14375.jpg  surprise
7064   test/surprise/9806.jpg  surprise
7065  test/surprise/35530.jpg  surprise

[7066 rows x 2 columns]


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img)
        features.append(img)
    features = np.array(features) ## neural networks (like Keras/TensorFlow) expect a single NumPy array
    features = features.reshape(len(features), 48,48,1)
    return features

In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [16]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [17]:
x_train = train_features/255.0
x_test = test_features/255.0

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [20]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])


In [21]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)


In [22]:
from tensorflow.keras.layers import BatchNormalization


In [23]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Flatten + Fully connected
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,743 (1.36 MB)

 Trainable params: 356,295 (1.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #update the weights smartly, following the loss landscape, to minimize the loss.

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


In [27]:
model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[earlystop])

Epoch 1/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 171s 371ms/step - accuracy: 0.2481 - loss: 2.0053 - val_accuracy: 0.3022 - val_loss: 3.1538
Epoch 2/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 202s 371ms/step - accuracy: 0.3619 - loss: 1.6212 - val_accuracy: 0.3808 - val_loss: 1.6332
Epoch 3/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 161s 356ms/step - accuracy: 0.4024 - loss: 1.5273 - val_accuracy: 0.4682 - val_loss: 1.3840
Epoch 4/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 161s 358ms/step - accuracy: 0.4296 - loss: 1.4675 - val_accuracy: 0.4492 - val_loss: 1.4488
Epoch 5/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 200s 354ms/step - accuracy: 0.4410 - loss: 1.4361 - val_accuracy: 0.5030 - val_loss: 1.3372
Epoch 6/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 204s 358ms/step - accuracy: 0.4692 - loss: 1.3809 - val_accuracy: 0.5018 - val_loss: 1.3112
Epoch 7/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 163s 361ms/step - accuracy: 0.4785 - loss: 1.3636 - val_accuracy: 0.5211 - val_loss: 1.2678
Epoch 8/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 192s 339ms/step - accuracy: 0.4830 -

In [28]:
model.save('best_emotion_model.h5')

In [29]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

221/221 ━━━━━━━━━━━━━━━━━━━━ 14s 63ms/step - accuracy: 0.5214 - loss: 1.2857
Test Accuracy: 0.5991
